In [1]:
import numpy as np
import pandas as pd 
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/home/minwook0008/bin/anaconda3/envs/MLAp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/minwook0008/bin/anaconda3/envs/MLAp/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
# tested in transformers==4.18.0, pytorch==1.7.1 
import torch
import transformers
torch.__version__, transformers.__version__

('2.0.1+cu117', '4.40.0')

In [3]:
device = torch.device("cuda")
device

device(type='cuda')

In [4]:
torch.cuda.is_available()

True

In [5]:
df_train = pd.read_csv('/home/minwook0008/please/OR2/dataset/train_stockemo.csv') 
df_val = pd.read_csv('/home/minwook0008/please/OR2/dataset/val_stockemo.csv')
df_test = pd.read_csv('/home/minwook0008/please/OR2/dataset/test_stockemo.csv')
df_test

id        date ticker   emo_label senti_label  \
0    100067  2020-01-08   AAPL   confusion     bearish   
1    100115  2020-01-10     BA     anxiety     bullish   
2    100175  2020-01-15   SBUX   confusion     bearish   
3    100234  2020-01-18   TSLA   confusion     bearish   
4    100268  2020-01-22     BA   confusion     bearish   
..      ...         ...    ...         ...         ...   
995  109996  2020-12-31   ABNB    optimism     bearish   
996  109997  2020-12-31   TSLA     disgust     bullish   
997  109998  2020-12-31   BABA   confusion     bullish   
998  109999  2020-12-31    CCL   amusement     bullish   
999  110000  2020-12-31   TSLA  excitement     bullish   

                                              original  \
0    $AAPL with all time highs again the market see...   
1         $BA about to explode 🚨\nKeep shorting please   
2    $SBUX TPO alot of volume at 91.74, thats after...   
3                 $TSLA down 6$ afterhours right now 🤔   
4    $BA can someone tell me why they haven’t downg...   
..                                                 ...   
995  $ABNB “sugar daddy puts.” Don’t mind me, I’m j...   
996  $TSLA \nGood news... now bears can get help wh...   
997  $BABA Who else is glad they sold in 240s yeste...   
998       $CCL $23 calls for .79 you know what to do 🥳   
999  $TSLA  🔥 closed out 15k in profit!Happy New Ye...   

                                             processed  
0    Apple with all time highs again the market see...  
1    Boeing about to explode  [police car light] \n...  
2    Starbucks TPO alot of volume at 91.74, thats a...  
3    Tesla down 6$ afterhours right now  [thinking ...  
4    Boeing can someone tell me why they haven’t do...  
..                                                 ...  
995  Airbnb “sugar daddy puts.” Don’t mind me, I’m ...  
996  Tesla \nGood news... now bears can get help wh...  
997  Alibaba Who else is glad they sold in 240s yes...  
998  Carnival $23 calls for .79 you know what to do...  
999  Tesla   [fire]  closed out 15k in profit!Happy...  

[1000 rows x 7 columns]

In [6]:
df_train = df_train.dropna(subset=['senti_label', 'processed'])
df_val = df_val.dropna(subset=['senti_label', 'processed'])
df_test = df_test.dropna(subset=['senti_label', 'processed'])

In [7]:
df_train

id        date ticker   emo_label senti_label  \
0     100001  2020-01-01   AMZN  excitement     bullish   
1     100002  2020-01-01   TSLA  excitement     bullish   
2     100003  2020-01-01   AAPL   confusion     bullish   
3     100004  2020-01-01   TSLA  excitement     bullish   
4     100005  2020-01-01   TSLA  excitement     bullish   
...      ...         ...    ...         ...         ...   
7995  109536  2020-08-06   AAPL    surprise     bullish   
7996  109541  2020-08-10   AAPL      belief     bullish   
7997  109561  2020-08-18   AMZN   ambiguous     bullish   
7998  109795  2020-10-01   TSLA    surprise     bullish   
7999  109840  2020-10-21   TSLA    surprise     bullish   

                                               original  \
0          $AMZN Dow futures up by 100 points already 🥳   
1     $TSLA Daddy's drinkin' eArly tonight! Here's t...   
2     $AAPL We’ll been riding since last December fr...   
3               $TSLA happy new year, 2020, everyone🍷🎉🙏   
4     $TSLA haha just a collection of greats..."Mars...   
...                                                 ...   
7995  BREAKING NEWS: $AAPL beats Elon Musk and Richa...   
7996  $AAPL why be a 2T giant and not dabble a littl...   
7997                  $AMZN - 3,500 jobs = $3,500 ??? 😁   
7998                  $TSLA $500c 10/2 at 36k volume 👀👀   
7999  $TSLA beat was more significant than I was exp...   

                                              processed  
0     Amazon Dow futures up by 100 points already  [...  
1     Tesla Daddy's drinkin' eArly tonight! Here's t...  
2     Apple We’ll been riding since last December fr...  
3     Tesla happy new year, 2020, everyone [wine gla...  
4     Tesla haha just a collection of greats..."Mars...  
...                                                 ...  
7995  BREAKING NEWS: Apple beats Elon Musk and Richa...  
7996  Apple why be a 2T giant and not dabble a littl...  
7997  Amazon - 3,500 jobs = $3,500 ???  [beaming fac...  
7998             Tesla $500c 10/2 at 36k volume  [eyes]  
7999  Tesla beat was more significant than I was exp...  

[8000 rows x 7 columns]

In [8]:
selected_columns = ['senti_label', 'processed']
df_train_pre = df_train[selected_columns]
df_val_pre = df_val[selected_columns]
df_test_pre = df_test[selected_columns]
df_test_pre.head()

senti_label                                          processed
0     bearish  Apple with all time highs again the market see...
1     bullish  Boeing about to explode  [police car light] \n...
2     bearish  Starbucks TPO alot of volume at 91.74, thats a...
3     bearish  Tesla down 6$ afterhours right now  [thinking ...
4     bearish  Boeing can someone tell me why they haven’t do...

In [9]:
df_train_pre['senti_label'] = df_train_pre['senti_label'].apply(lambda x: 2 if x == 'bullish' else (1 if x == 'neutral' else 0))
df_val_pre['senti_label'] = df_train_pre['senti_label'].apply(lambda x: 2 if x == 'bullish' else (1 if x == 'neutral' else 0))
df_test_pre['senti_label'] = df_train_pre['senti_label'].apply(lambda x: 2 if x == 'bullish' else (1 if x == 'neutral' else 0))

/tmp/ipykernel_3255237/2219857626.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_pre['senti_label'] = (df_train_pre['senti_label'] == 'bullish').astype(int) * 2
/tmp/ipykernel_3255237/2219857626.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val_pre['senti_label'] = (df_val_pre['senti_label'] == 'bullish').astype(int) * 2
/tmp/ipykernel_3255237/2219857626.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [10]:
df_test_pre.head()

senti_label                                          processed
0            0  Apple with all time highs again the market see...
1            2  Boeing about to explode  [police car light] \n...
2            0  Starbucks TPO alot of volume at 91.74, thats a...
3            0  Tesla down 6$ afterhours right now  [thinking ...
4            0  Boeing can someone tell me why they haven’t do...

### load FinBERT pretrained model
The pretrained FinBERT model path on Huggingface is https://huggingface.co/yiyanghkust/finbert-pretrain

In [11]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3).to(device)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

/home/minwook0008/bin/anaconda3/envs/MLAp/lib/python3.9/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [21]:
df_train_pre.rename(columns={'senti_label': 'label'}, inplace=True)
df_val_pre.rename(columns={'senti_label': 'label'}, inplace=True)
df_test_pre.rename(columns={'senti_label': 'label'}, inplace=True)
df_train_pre

/tmp/ipykernel_3255237/139600951.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_pre.rename(columns={'senti_label': 'label'}, inplace=True)
/tmp/ipykernel_3255237/139600951.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val_pre.rename(columns={'senti_label': 'label'}, inplace=True)
/tmp/ipykernel_3255237/139600951.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_pre.rename(columns={'senti_label': 'label'

label                                          processed
0         2  Amazon Dow futures up by 100 points already  [...
1         2  Tesla Daddy's drinkin' eArly tonight! Here's t...
2         2  Apple We’ll been riding since last December fr...
3         2  Tesla happy new year, 2020, everyone [wine gla...
4         2  Tesla haha just a collection of greats..."Mars...
...     ...                                                ...
7995      2  BREAKING NEWS: Apple beats Elon Musk and Richa...
7996      2  Apple why be a 2T giant and not dabble a littl...
7997      2  Amazon - 3,500 jobs = $3,500 ???  [beaming fac...
7998      2             Tesla $500c 10/2 at 36k volume  [eyes]
7999      2  Tesla beat was more significant than I was exp...

[8000 rows x 2 columns]

### prepare dataset for fine-tuning

In [22]:
dataset_train = Dataset.from_pandas(df_train_pre)
dataset_val = Dataset.from_pandas(df_val_pre)
dataset_test = Dataset.from_pandas(df_test_pre)

dataset_train = dataset_train.map(lambda e: tokenizer(e['processed'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['processed'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['processed'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])


Map: 100%|█████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2956.36 examples/s]


In [23]:
dataset_train

Dataset({
    features: ['label', 'processed', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8000
})

In [24]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,                   
        args=args,                 
        train_dataset=dataset_train,       
        eval_dataset=dataset_val,          
        compute_metrics=compute_metrics
)

trainer.train() 

TrainOutput(global_step=1250, training_loss=0.35364760437011716, metrics={'train_runtime': 236.8446, 'train_samples_per_second': 168.887, 'train_steps_per_second': 5.278, 'total_flos': 2631134177280000.0, 'train_loss': 0.35364760437011716, 'epoch': 5.0})

### evaluate on testing set

In [62]:
model.eval()
trainer.predict(dataset_test).metrics

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, __index_level_0__. If sentence, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 32


{'test_loss': 0.32989031076431274,
 'test_accuracy': 0.881,
 'test_runtime': 1.4364,
 'test_samples_per_second': 696.197,
 'test_steps_per_second': 5.57}

### save the fine-tuned model

In [63]:
trainer.save_model('finbert-sentiment/')

Saving model checkpoint to finbert-sentiment/
Configuration saved in finbert-sentiment/config.json
Model weights saved in finbert-sentiment/pytorch_model.bin
